# ML-Webservice: Exposing webservice

## Libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')
from werkzeug.wrappers import Request, Response
from flask import Flask, jsonify
import json
import pickle
import Orange
import platform
print("Python:" + platform.python_version())
print("Orange:" + Orange.version.version)

Python:3.8.3
Orange:3.26.0


## Load Models

In [2]:
models=[]
#Load model
saved_model=["model/neural-network.pkcls","model/logistic-regression.pkcls","model/gradient-boosting.pkcls"]
saved_model=["model/model-ann.pkcls","model/model-lr.pkcls","model/model-gb.pkcls"]
for fileName in saved_model:
    print ('Loading file:',fileName)
    with open(fileName, "rb") as f:
        models.append(pickle.load(f))

Loading file: model/model-ann.pkcls
Loading file: model/model-lr.pkcls
Loading file: model/model-gb.pkcls


## json_flat_array

In [3]:
def json_flat_array(y):
    arr=[]
    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                arr.append(x[a])            
    flatten(y)
    return arr

#json_flat_array (dataJSON)

## PredictionByVoting

In [4]:
def predictByVoting(input_for_prediction):
    votes=[]
    jsonObj = []
    
    data_predict= (json_flat_array (input_for_prediction)) # input_for_prediction 
    for mdl in models:
        pred_ind = int(mdl(data_predict))
        prob = mdl(data_predict, mdl.Probs).round(6) 
        votes.append(pred_ind)
        #print ("model:"+mdl.name+",", "outcome:"+str(pred_ind)+", ","probability:"+str(prob))
        js= { "model" : mdl.name , "outcome" : str(pred_ind), "probability" :str(prob) }
        jsonObj.append(js)
 
    #print("votes:"+ str(votes))

    y=0;n=0;
    for c in votes:
        if (c==1):
            y=y+1
        else:
            n=n+1

    
    #print ("\nBased on maximum votes, the final outcome is:")
    if (y>n):
        concludedOutcome="Positive"
    else:
        concludedOutcome="Nagative"
    
    responseObj={"concluded" : concludedOutcome}
    responseObj.update ({"details" : jsonObj})
    
    return concludedOutcome,json.dumps(responseObj)
        

## Unit Test

In [5]:
# Nagative
dataJSON = {
    'age' : 45.0,
    'job' : 'entrepreneur', 
    'marital' : 'married', 
    'education' : 'university.degree', 
    'default' : 'unknown', 
    'balance' : 'yes', 
    'housing' : 'yes', 
    'contact' : 'cellular', 
    'month' : 'aug',
    'day_of_week' : 'mon',
    'duration' :  38.0, 
    'campaign' : 2.0, 
    'pdays' : 0,
    'anyvalue' : 0.0, 
    'previous' : 'nonexistent', 
    'poutcome' : 1.4, 
    'emp.var.rate' : 93.444, 
    'cons.price.idx' : -36.1, 
    'cons.conf.idx' : 4.965, 
    'euribor3m' :5228.1
}

In [6]:
# Positive
dataJSON = {
    'age' : 50.0,
    'job' : 'blue-collar', 
    'marital' : 'married', 
    'education' : 'basic.4y', 
    'default' : 'no', 
    'balance' : 'no', 
    'housing' : 'yes', 
    'contact' : 'cellular', 
    'month' : 'jul',
    'day_of_week' : 'tue',
    'duration' :  849.0, 
    'campaign' : 1.0, 
    'pdays' : 0,
    'anyvalue' : 0.0,
    'previous' : 'nonexistent', 
    'poutcome' : 1.4, 
    'emp.var.rate' : 93.918, 
    'cons.price.idx' : -42.7, 
    'cons.conf.idx' : 4.961, 
    'euribor3m' :5228.1
}

In [7]:
# Predict any unknow instance
concludedOutcome,jsonObj= predictByVoting(dataJSON)
print(concludedOutcome)
print (jsonObj)

Positive
{"concluded": "Positive", "details": [{"model": "Artificial Neural Network", "outcome": "1", "probability": "[0.433643 0.566357]"}, {"model": "Logistic Regression", "outcome": "0", "probability": "[0.559669 0.440331]"}, {"model": "Gradient Boosting", "outcome": "1", "probability": "[0.357833 0.642167]"}]}


## Exposing as Service

In [8]:
from flask import Flask, request, jsonify
app = Flask(__name__)

@app.route("/predict", methods=['GET', 'POST'])
def predict():
    inputJSON = request.json
    print("input:")
    print(inputJSON)
    print("output:")
    finalOutcome,jsonObj= predictByVoting(inputJSON)
    print ("Concluded outcome:"+finalOutcome+"\n")
    print(jsonObj)
    
    return jsonify(jsonObj)

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Oct/2021 19:07:52] "POST /predict HTTP/1.1" 200 -


input:
{'age': 50.0, 'job': 'blue-collar', 'marital': 'married', 'education': 'basic.4y', 'default': 'no', 'balance': 'no', 'housing': 'yes', 'contact': 'cellular', 'month': 'jul', 'day_of_week': 'tue', 'duration': 849.0, 'campaign': 1.0, 'pdays': 0, 'anyvalue': 0.0, 'previous': 'nonexistent', 'poutcome': 1.4, 'emp.var.rate': 93.918, 'cons.price.idx': -42.7, 'cons.conf.idx': 4.961, 'euribor3m': 5228.1}
output:
Concluded outcome:Positive

{"concluded": "Positive", "details": [{"model": "Artificial Neural Network", "outcome": "1", "probability": "[0.433643 0.566357]"}, {"model": "Logistic Regression", "outcome": "0", "probability": "[0.559669 0.440331]"}, {"model": "Gradient Boosting", "outcome": "1", "probability": "[0.357833 0.642167]"}]}


127.0.0.1 - - [16/Oct/2021 19:08:47] "POST /predict HTTP/1.1" 200 -


input:
{'age': 50.0, 'job': 'blue-collar', 'marital': 'married', 'education': 'basic.4y', 'default': 'no', 'balance': 'no', 'housing': 'yes', 'contact': 'cellular', 'month': 'jul', 'day_of_week': 'tue', 'duration': 849.0, 'campaign': 1.0, 'pdays': 0, 'anyvalue': 0.0, 'previous': 'nonexistent', 'poutcome': 1.4, 'emp.var.rate': 93.918, 'cons.price.idx': -42.7, 'cons.conf.idx': 4.961, 'euribor3m': 5228.1}
output:
Concluded outcome:Positive

{"concluded": "Positive", "details": [{"model": "Artificial Neural Network", "outcome": "1", "probability": "[0.433643 0.566357]"}, {"model": "Logistic Regression", "outcome": "0", "probability": "[0.559669 0.440331]"}, {"model": "Gradient Boosting", "outcome": "1", "probability": "[0.357833 0.642167]"}]}
